# T1 - Reprise du modèle défini à la partie 1

Notre modèle dimensionnel révisé comprend :
- 2 tables de faits : FACT_SALES et FACT_INVENTORY
- 5 dimensions : DIM_TIME, DIM_PRODUCT, DIM_CUSTOMER, DIM_GEOGRAPHY, DIM_DEPARTMENT

Vérification de la faisabilité avec les données disponibles :
- Pour FACT_SALES : Nous avons orders.csv et order_details.csv qui contiennent les données nécessaires
- Pour FACT_INVENTORY : Nous devrons dériver cette table à partir des ventes et des capacités de stockage
- Pour les dimensions : Nous avons les fichiers correspondants (customers.csv, products.csv, etc.)

Le modèle est donc réalisable avec les données disponibles, bien que certaines mesures devront être calculées ou estimées.

# T2 - Importation des fichiers CSV et inspection des structures
Commençons par importer les fichiers CSV et examiner leur structure :

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [2]:
# Importation des fichiers CSV
addresses_df = pd.read_csv('data/addresses.csv')
customers_df = pd.read_csv('data/customers.csv')
departments_df = pd.read_csv('data/departments.csv')
order_details_df = pd.read_csv('data/order_details.csv')
orders_df = pd.read_csv('data/orders.csv')
products_df = pd.read_csv('data/products.csv')

In [3]:
# Inspection des structures
print("=== Aperçu de addresses_df ===")
print(addresses_df.info())
print(addresses_df.head())
print("\n=== Aperçu de customers_df ===")
print(customers_df.info())
print(customers_df.head())
print("\n=== Aperçu de departments_df ===")
print(departments_df.info())
print(departments_df.head())
print("\n=== Aperçu de order_details_df ===")
print(order_details_df.info())
print(order_details_df.head())
print("\n=== Aperçu de orders_df ===")
print(orders_df.info())
print(orders_df.head())
print("\n=== Aperçu de products_df ===")
print(products_df.info())
print(products_df.head())

=== Aperçu de addresses_df ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4910 entries, 0 to 4909
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Address ID   4910 non-null   object
 1   Customer ID  4910 non-null   object
 2   Country      4910 non-null   object
 3   City         4910 non-null   object
 4   State        4910 non-null   object
 5   Postal Code  4910 non-null   int64 
 6   Region       4910 non-null   object
dtypes: int64(1), object(6)
memory usage: 268.6+ KB
None
       Address ID Customer ID        Country             City           State  \
0  CG-12520_42420    CG-12520  United States        Henderson        Kentucky   
1  DV-13045_90036    DV-13045  United States      Los Angeles      California   
2  SO-20335_33311    SO-20335  United States  Fort Lauderdale         Florida   
3  BH-11710_90032    BH-11710  United States      Los Angeles      California   
4  AA-10480_28027    AA-10480  U

In [4]:
# Vérification des valeurs manquantes
print("\n=== Valeurs manquantes ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print(f"{df_name} contient des valeurs manquantes:")
        print(missing_values[missing_values > 0])
    else:
        print(f"{df_name} ne contient pas de valeurs manquantes")


=== Valeurs manquantes ===
addresses_df ne contient pas de valeurs manquantes
customers_df ne contient pas de valeurs manquantes
departments_df ne contient pas de valeurs manquantes
order_details_df ne contient pas de valeurs manquantes
orders_df ne contient pas de valeurs manquantes
products_df ne contient pas de valeurs manquantes


In [6]:
# Vérification des doublons
print("\n=== Doublons ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    duplicates = df.duplicated().sum()
    print(f"{df_name} contient {duplicates} doublons")


=== Doublons ===
addresses_df contient 0 doublons
customers_df contient 0 doublons
departments_df contient 0 doublons
order_details_df contient 0 doublons
orders_df contient 0 doublons
products_df contient 0 doublons


In [15]:
# Vérification des types de données
print("\n=== Types de données ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    print(f"{df_name} types:")
    print(df.dtypes)


=== Types de données ===
addresses_df types:
Address ID     object
Customer ID    object
Country        object
City           object
State          object
Postal Code     int64
Region         object
dtype: object
customers_df types:
Customer ID      object
Customer Name    object
Segment          object
dtype: object
departments_df types:
Department          object
City                object
Storage Capacity     int64
dtype: object
order_details_df types:
Order ID       object
Product ID     object
Sales         float64
Quantity        int64
Profit        float64
dtype: object
orders_df types:
Order ID               object
Order Date     datetime64[ns]
Ship Date      datetime64[ns]
Customer ID            object
Address ID             object
dtype: object
products_df types:
Product ID      object
Department      object
Sub-Category    object
Product Name    object
dtype: object


Après cette inspection, voici ce que nous pouvons relever :
- Contrairement à nos suppositions, aucun des dataframes ne contient de valeurs manquantes, ce qui signifie que les données sont d'excellente qualité.
- Aucun des dataframes ne contient de doublons, ce qui simplifie le processus de nettoyage.
- Les dates dans orders_df (Order Date, Ship Date) sont au format string (object) et doivent être converties en format datetime pour permettre des opérations temporelles.
- Les autres types de données semblent appropriés (les valeurs numériques sont déjà en int64 ou float64).

# T3 - Nettoyage des données
Maintenant, procédons au nettoyage des données pour chaque dataframe :

In [17]:
# Vérification et traitement des valeurs manquantes (il ne semble pas y en avoir, mais on ne sait jamais)
# Pour addresses_df
if addresses_df.isnull().sum().sum() > 0: # le premier .sum() donne le nombre de isNull par colonne, le second fait la somme de ces isNull
    # Remplir les valeurs manquantes ou supprimer les lignes selon le contexte
    addresses_df = addresses_df.dropna(subset=['Customer ID', 'City', 'State', 'Region'])
    # Pour les autres colonnes, on peut remplir avec des valeurs par défaut
    addresses_df['Postal Code'].fillna('Unknown', inplace=True)

# Pour customers_df
if customers_df.isnull().sum().sum() > 0:
    # Les informations client doivent être complètes pour être exploitables, donc on supprime les lignes avec des valeurs manquantes
    customers_df = customers_df.dropna()

# Pour order_details_df
if order_details_df.isnull().sum().sum() > 0:
    # Vérifier les colonnes avec des valeurs manquantes
    # Pour les valeurs numériques, on peut remplacer par 0 ou la moyenne
    order_details_df['Quantity'].fillna(0, inplace=True)
    order_details_df['Sales'].fillna(0, inplace=True)
    order_details_df['Profit'].fillna(0, inplace=True)
    # Supprimer les lignes où Product ID est manquant
    order_details_df = order_details_df.dropna(subset=['Product ID'])

# Pour products_df
if products_df.isnull().sum().sum() > 0:
    # Les informations produit sont essentielles, donc on supprime les lignes avec des valeurs manquantes
    products_df = products_df.dropna()

In [11]:
# Suppression des doublons (les outputs de T2 n'en montrent pas, mais on ne sait jamais)
addresses_df = addresses_df.drop_duplicates()
customers_df = customers_df.drop_duplicates()
departments_df = departments_df.drop_duplicates()
order_details_df = order_details_df.drop_duplicates()
orders_df = orders_df.drop_duplicates()
products_df = products_df.drop_duplicates()

In [8]:
# Conversion des dates dans orders_df
orders_df['Order Date'] = pd.to_datetime(orders_df['Order Date'])
orders_df['Ship Date'] = pd.to_datetime(orders_df['Ship Date'])

In [20]:
# Normalisation (nettoyage et uniformisation) des valeurs textuelles
# Pour products_df
products_df['Department'] = products_df['Department'].str.strip().str.title()
products_df['Sub-Category'] = products_df['Sub-Category'].str.strip().str.title()
products_df['Product Name'] = products_df['Product Name'].str.strip()

# Pour customers_df
customers_df['Segment'] = customers_df['Segment'].str.strip().str.title()
customers_df['Customer Name'] = customers_df['Customer Name'].str.strip()

# Pour addresses_df
addresses_df['City'] = addresses_df['City'].str.strip().str.title()
addresses_df['State'] = addresses_df['State'].str.strip().str.title()
addresses_df['Region'] = addresses_df['Region'].str.strip().str.title()
addresses_df['Country'] = addresses_df['Country'].str.strip().str.title()

# Pour departments_df
departments_df['Department'] = departments_df['Department'].str.strip().str.title()
departments_df['City'] = departments_df['City'].str.strip().str.title()

In [14]:
# Vérification des données après nettoyage
print("\n=== Données après nettoyage ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    print(f"{df_name} shape: {df.shape}")
    print(f"Valeurs manquantes: {df.isnull().sum().sum()}")
    print(f"Doublons: {df.duplicated().sum()}")


=== Données après nettoyage ===
addresses_df shape: (4910, 7)
Valeurs manquantes: 0
Doublons: 0
customers_df shape: (793, 3)
Valeurs manquantes: 0
Doublons: 0
departments_df shape: (3, 3)
Valeurs manquantes: 0
Doublons: 0
order_details_df shape: (9993, 5)
Valeurs manquantes: 0
Doublons: 0
orders_df shape: (5009, 5)
Valeurs manquantes: 0
Doublons: 0
products_df shape: (1862, 4)
Valeurs manquantes: 0
Doublons: 0


# 